In [4]:
!pip install gensim

  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.25
    Uninstalling Cython-0.29.25:
      Successfully uninstalled Cython-0.29.25


In [22]:
import csv
import pandas as pd
import numpy as np

import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import plotly.express as px

In [9]:
file_name = "scopus.csv"
df_scopus = pd.read_csv(file_name)
df_scopus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Authors            908 non-null    object 
 1   Author(s) ID       907 non-null    object 
 2   Title              908 non-null    object 
 3   Year               908 non-null    int64  
 4   Source title       908 non-null    object 
 5   Volume             871 non-null    object 
 6   Issue              802 non-null    object 
 7   Art. No.           144 non-null    object 
 8   Page start         756 non-null    object 
 9   Page end           756 non-null    object 
 10  Page count         1 non-null      float64
 11  Cited by           724 non-null    float64
 12  DOI                803 non-null    object 
 13  Link               908 non-null    object 
 14  Abstract           908 non-null    object 
 15  Author Keywords    783 non-null    object 
 16  Index Keywords     221 non

In [10]:
relevant_columns = {"index": "DOI", "LSA": "Title  Abstract  Author Keywords  Index Keywords".split("  ")}

In [11]:
df_docs = df_scopus[["DOI"]+relevant_columns["LSA"]]
df_docs = df_docs.set_index("DOI")
df_docs["Abstract"] = df_docs["Abstract"].replace({"[No abstract available]": np.nan})
df_docs = df_docs.fillna("")
df_docs

,Title,Abstract,Author Keywords,Index Keywords
DOI,,,,
10.1108/IJSHE-12-2020-0484,"Students’ learning sustainability – implicit, ...",Purpose: This study aims to understand better ...,Higher education institutions (HEIs); Sustaina...,
10.1057/s41599-022-01129-0,Going beyond the AHA! moment: insight discover...,"In this paper, we develop and apply the concep...",,
10.1186/s12909-022-03308-8,An online Delphi study to investigate the comp...,Background: Several competency frameworks are ...,Allied health disciplines; CanMEDS; Continuous...,article; audiology; Belgium; clinical article;...
10.1186/s12909-022-03283-0,How do medical students learn conceptual knowl...,Background: Acquiring medical knowledge is a k...,Learning difficulties; Learning techniques; Me...,adult; article; controlled study; curriculum; ...
10.1186/s12909-022-03259-0,How medical students co-regulate their learnin...,Background: Self-regulated learning is a key c...,Clinical clerkships; Co-regulated learning; Me...,clinical education; human; learning; medical e...
...,...,...,...,...
10.1080/0309877920160311,“Quality Management’ or “The Educative Workpla...,Competence-based education has been the topic ...,,
10.1016/0261-5177(92)90051-8,The implications of competence based education...,,,
10.1080/00048623.1992.10754796,Information Literacy — A Core Competency,The Finn Report highlights the need for young ...,,


In [12]:
df_docs["doc"] = df_docs.apply(lambda x: ". ".join(x.values), axis=1)
df_docs

,Title,Abstract,Author Keywords,Index Keywords,doc
DOI,,,,,
10.1108/IJSHE-12-2020-0484,"Students’ learning sustainability – implicit, ...",Purpose: This study aims to understand better ...,Higher education institutions (HEIs); Sustaina...,,"Students’ learning sustainability – implicit, ..."
10.1057/s41599-022-01129-0,Going beyond the AHA! moment: insight discover...,"In this paper, we develop and apply the concep...",,,Going beyond the AHA! moment: insight discover...
10.1186/s12909-022-03308-8,An online Delphi study to investigate the comp...,Background: Several competency frameworks are ...,Allied health disciplines; CanMEDS; Continuous...,article; audiology; Belgium; clinical article;...,An online Delphi study to investigate the comp...
10.1186/s12909-022-03283-0,How do medical students learn conceptual knowl...,Background: Acquiring medical knowledge is a k...,Learning difficulties; Learning techniques; Me...,adult; article; controlled study; curriculum; ...,How do medical students learn conceptual knowl...
10.1186/s12909-022-03259-0,How medical students co-regulate their learnin...,Background: Self-regulated learning is a key c...,Clinical clerkships; Co-regulated learning; Me...,clinical education; human; learning; medical e...,How medical students co-regulate their learnin...
...,...,...,...,...,...
10.1080/0309877920160311,“Quality Management’ or “The Educative Workpla...,Competence-based education has been the topic ...,,,“Quality Management’ or “The Educative Workpla...
10.1016/0261-5177(92)90051-8,The implications of competence based education...,,,,The implications of competence based education...
10.1080/00048623.1992.10754796,Information Literacy — A Core Competency,The Finn Report highlights the need for young ...,,,Information Literacy — A Core Competency. The ...


In [230]:
for ix in df_docs.iloc[2].index:
    print(ix)
    print(df_docs.iloc[2][ix], '\n')

Title
An online Delphi study to investigate the completeness of the CanMEDS Roles and the relevance, formulation, and measurability of their key competencies within eight healthcare disciplines in Flanders 

Abstract
Background: Several competency frameworks are being developed to support competency-based education (CBE). In medical education, extensive literature exists about validated competency frameworks for example, the CanMEDS competency framework. In contrast, comparable literature is limited in nursing, midwifery, and allied health disciplines. Therefore, this study aims to investigate (1) the completeness of the CanMEDS Roles, and (2) the relevance, formulation, and measurability of the CanMEDS key competencies in nursing, midwifery, and allied health disciplines. If the competency framework is validated in different educational programs, opportunities to support CBE and interprofessional education/collaboration can be created. Methods: A three-round online Delphi study was co

In [232]:
" ".join(df_docs.iloc[2]["doc_clean"])

'onlin delphi studi investig complet canm role relev formul measur kei compet healthcar disciplin flander background compet framework develop support compet base educ cbe medic educ extens literatur exist valid compet framework exampl canm compet framework contrast compar literatur limit nurs midwiferi alli health disciplin therefor studi aim investig complet canm role relev formul measur canm kei compet nurs midwiferi alli health disciplin compet framework valid differ educ program opportun support cbe interprofession educ collabor creat method three round onlin delphi studi conduct respect expert rate role kei compet expert came non univers healthcar disciplin flander belgium audiolog dental hygien midwiferi nurs occup therapi podiatri speech therapi expert answer ye role likert type scale kei compet agreement percentag analyz quantit consensu attain expert score posit round on expert add remark qualit analyz induct content analysi result round on consensu complet role relev formul m

In [13]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, preprocess_string, strip_short, stem_text

In [14]:
def preprocess(text):
    CUSTOM_FILTERS = [lambda x: x.lower(), 
                                remove_stopwords, 
                                strip_punctuation, 
                                strip_short, 
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    return text

In [15]:
df_docs['doc_clean'] = df_docs['doc'].apply(lambda x: preprocess(x))

In [16]:
df_docs

,Title,Abstract,Author Keywords,Index Keywords,doc,doc_clean
DOI,,,,,,
10.1108/IJSHE-12-2020-0484,"Students’ learning sustainability – implicit, ...",Purpose: This study aims to understand better ...,Higher education institutions (HEIs); Sustaina...,,"Students’ learning sustainability – implicit, ...","[students’, learn, sustain, implicit, explicit..."
10.1057/s41599-022-01129-0,Going beyond the AHA! moment: insight discover...,"In this paper, we develop and apply the concep...",,,Going beyond the AHA! moment: insight discover...,"[go, aha, moment, insight, discoveri, transdis..."
10.1186/s12909-022-03308-8,An online Delphi study to investigate the comp...,Background: Several competency frameworks are ...,Allied health disciplines; CanMEDS; Continuous...,article; audiology; Belgium; clinical article;...,An online Delphi study to investigate the comp...,"[onlin, delphi, studi, investig, complet, canm..."
10.1186/s12909-022-03283-0,How do medical students learn conceptual knowl...,Background: Acquiring medical knowledge is a k...,Learning difficulties; Learning techniques; Me...,adult; article; controlled study; curriculum; ...,How do medical students learn conceptual knowl...,"[medic, student, learn, conceptu, knowledg, hi..."
10.1186/s12909-022-03259-0,How medical students co-regulate their learnin...,Background: Self-regulated learning is a key c...,Clinical clerkships; Co-regulated learning; Me...,clinical education; human; learning; medical e...,How medical students co-regulate their learnin...,"[medic, student, regul, learn, clinic, clerksh..."
...,...,...,...,...,...,...
10.1080/0309877920160311,“Quality Management’ or “The Educative Workpla...,Competence-based education has been the topic ...,,,“Quality Management’ or “The Educative Workpla...,"[“qualiti, management’, “the, educ, workplace’..."
10.1016/0261-5177(92)90051-8,The implications of competence based education...,,,,The implications of competence based education...,"[implic, compet, base, educ, train, programm, ..."
10.1080/00048623.1992.10754796,Information Literacy — A Core Competency,The Finn Report highlights the need for young ...,,,Information Literacy — A Core Competency. The ...,"[inform, literaci, core, compet, finn, report,..."


In [17]:
import json
  
f = open('citation_net.json', encoding='utf-8-sig')
data = json.load(f)
network = data["network"]

In [23]:
network_items = pd.DataFrame(network["items"])
network_items["DOI"] = network_items["url"].apply(lambda x: x.replace("https://doi.org/", "") if str(x) != "nan" else np.nan)
network_items = network_items.set_index("DOI")

In [24]:
network_items

,id,label,description,url,x,y,cluster,weights,scores
DOI,,,,,,,,,
10.1108/ijshe-12-2020-0484,1,alm k. (2022),<table><tr><td>Authors:</td><td>alm k.; beery ...,https://doi.org/10.1108/ijshe-12-2020-0484,0.7611,-0.0492,1,"{'Links': 4.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm...."
10.1057/s41599-022-01047-1,7,škrinjarić b. (2022),<table><tr><td>Authors:</td><td>škrinjarić b.<...,https://doi.org/10.1057/s41599-022-01047-1,0.6201,0.2079,4,"{'Links': 5.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm...."
10.14201/teri.25394,10,leal m.s. (2022),<table><tr><td>Authors:</td><td>leal m.s.; góm...,https://doi.org/10.14201/teri.25394,-0.3718,0.2822,3,"{'Links': 1.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm...."
10.1108/ijshe-12-2020-0506,11,birdman j. (2022),<table><tr><td>Authors:</td><td>birdman j.; wi...,https://doi.org/10.1108/ijshe-12-2020-0506,0.8060,-0.1802,1,"{'Links': 5.0, 'Citations': 1.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 1.0, 'Norm...."
10.3390/su14094916,12,venn r. (2022),<table><tr><td>Authors:</td><td>venn r.; perez...,https://doi.org/10.3390/su14094916,0.7053,-0.0819,1,"{'Links': 6.0, 'Citations': 0.0, 'Norm. citati...","{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm...."
...,...,...,...,...,...,...,...,...,...
10.1111/1467-9752.00070,880,hyland t. (1997),<table><tr><td>Authors:</td><td>hyland t.</td>...,https://doi.org/10.1111/1467-9752.00070,1.2992,-0.0930,1,"{'Links': 2.0, 'Citations': 55.0, 'Norm. citat...","{'Pub. year': 1997.0, 'Citations': 55.0, 'Norm..."
10.1108/00400919610117380,882,foot s. (1996),<table><tr><td>Authors:</td><td>foot s.; meggi...,https://doi.org/10.1108/00400919610117380,-1.0877,-0.3091,2,"{'Links': 1.0, 'Citations': 4.0, 'Norm. citati...","{'Pub. year': 1996.0, 'Citations': 4.0, 'Norm...."
10.1111/j.1467-9752.1996.tb00406.x,885,bridges d. (1996),<table><tr><td>Authors:</td><td>bridges d.</td...,https://doi.org/10.1111/j.1467-9752.1996.tb004...,0.4219,0.1045,4,"{'Links': 1.0, 'Citations': 18.0, 'Norm. citat...","{'Pub. year': 1996.0, 'Citations': 18.0, 'Norm..."


In [25]:
sub_df_docs = df_docs.copy().reset_index()
sub_df_docs = sub_df_docs.merge(network_items.reset_index(), on="DOI", how="inner").dropna()

In [136]:
table_hist = pd.DataFrame()
table_hist["Frequency"] = sub_df_docs["cluster"].value_counts().values
table_hist["Cluster"] = [str(i) for i in range(1, 6+1)]

color_discrete_sequence = ['#d63f4b', "#58bd5b", "#5599c3", "#bcbe4f", "#996fc0", "#5cccd9"]

fig = px.bar(table_hist, x="Cluster", y="Frequency", color="Cluster", title="Frecuency of clusters (LinLog/mod.)", color_discrete_sequence=color_discrete_sequence)
fig.update_layout(
    font=dict(
        size=18
    )
)
fig.show()

In [39]:
table_hist

,Frecuency,Cluster
0,52,1
1,39,2
2,24,3
3,20,4
4,9,5
5,6,6


In [57]:
from gensim import corpora
from gensim import models

In [207]:
cluster_id = 4
df_docs_cluster = sub_df_docs[sub_df_docs["cluster"] == cluster_id]
corpus = df_docs_cluster['doc_clean']
dictionary = corpora.Dictionary(corpus)

bow = [dictionary.doc2bow(text) for text in corpus]

tfidf = models.TfidfModel(bow)
corpus_tfidf = tfidf[bow]

In [208]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

In [209]:
%%time
for i in range(2, 16):
    lsi = LsiModel(corpus_tfidf, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df_docs_cluster['doc_clean'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} topics: {}'.format(i, coherence_score))

Coherence score with 2 topics: 0.3328857000677681
Coherence score with 3 topics: 0.32591682441435993
Coherence score with 4 topics: 0.3424467180114916
Coherence score with 5 topics: 0.3540608142587526
Coherence score with 6 topics: 0.3622095198149855
Coherence score with 7 topics: 0.3655890380984568
Coherence score with 8 topics: 0.5095417672677052
Coherence score with 9 topics: 0.3681209003274659
Coherence score with 10 topics: 0.39429464388462954
Coherence score with 11 topics: 0.4053669224076252
Coherence score with 12 topics: 0.39935698887493415
Coherence score with 13 topics: 0.3967510190539164
Coherence score with 14 topics: 0.39966511733074833
Coherence score with 15 topics: 0.40687805501060237
Wall time: 24.9 s


In [217]:
num_topics = 8
lsi = LsiModel(corpus_tfidf, num_topics=num_topics, id2word=dictionary)

In [218]:
corpus_lsi = lsi[bow]
scores = [[] for _ in range(num_topics)]
for doc in corpus_lsi:
    for k in range(num_topics):
        scores[k].append(round(doc[k][1],2))

df_topic = df_docs_cluster.copy()
for k in range(num_topics):
    df_topic[f'score_topic_{k}'] = scores[k]

df_topic['Topic']= df_topic[[f'score_topic_{k}' for k in range(num_topics)]].apply(lambda x: x.argmax(), axis=1)
df_topic

,DOI,Title,Abstract,Author Keywords,Index Keywords,doc,doc_clean,id,label,description,...,scores,score_topic_0,score_topic_1,score_topic_2,score_topic_3,score_topic_4,score_topic_5,score_topic_6,score_topic_7,Topic
0,10.1057/s41599-022-01047-1,Competence-based approaches in organizational ...,"With an increasing pace of digitalization, aut...",,,Competence-based approaches in organizational ...,"[compet, base, approach, organiz, individu, co...",7,škrinjarić b. (2022),<table><tr><td>Authors:</td><td>škrinjarić b.<...,...,"{'Pub. year': 2022.0, 'Citations': 0.0, 'Norm....",4.47,-1.19,3.81,-2.88,2.28,-3.30,3.83,-0.65,0
462,10.30880/jtet.2020.12.04.003,Validation of the chefs’ key competencies ques...,The chef is considered a success factor for cu...,CFA; Chef; Core competencies; Culinary experti...,,Validation of the chefs’ key competencies ques...,"[valid, chefs’, kei, compet, questionnair, cul...",254,mahfud t. (2020),<table><tr><td>Authors:</td><td>mahfud t.; nug...,...,"{'Pub. year': 2020.0, 'Citations': 0.0, 'Norm....",6.55,-0.28,-2.31,-7.59,-11.03,6.38,-1.07,4.62,0
480,10.1080/09639284.2018.1476895,Audit workplace simulations as a methodology t...,This paper describes an audit workplace simula...,communication skills; competence-based educati...,,Audit workplace simulations as a methodology t...,"[audit, workplac, simul, methodolog, increas, ...",397,bautista-mesa r. (2018),<table><tr><td>Authors:</td><td>bautista-mesa ...,...,"{'Pub. year': 2018.0, 'Citations': 10.0, 'Norm...",4.04,-1.08,2.29,-1.92,-1.36,3.46,-3.53,-10.53,0
481,10.3390/su10041155,Exercise on transdisciplinarity: Lessons from ...,Sustainability science emerged as a new academ...,Aging society; Competencies; Empathetic compet...,academic research; aging population; education...,Exercise on transdisciplinarity: Lessons from ...,"[exercis, transdisciplinar, lesson, field, bas...",401,kudo s. (2018),<table><tr><td>Authors:</td><td>kudo s.; mursa...,...,"{'Pub. year': 2018.0, 'Citations': 4.0, 'Norm....",9.83,-0.44,5.43,0.40,-3.87,-1.55,13.27,-6.34,6
491,10.1080/13540602.2015.1082728,Evaluation criteria for competency-based sylla...,"In recent decades, higher education institutio...",Chile; competence-based curriculum; criteria; ...,,Evaluation criteria for competency-based sylla...,"[evalu, criteria, compet, base, syllabi, chile...",503,jerez o. (2016),<table><tr><td>Authors:</td><td>jerez o.; vale...,...,"{'Pub. year': 2016.0, 'Citations': 4.0, 'Norm....",8.12,-4.71,-3.53,0.20,7.17,-2.21,-0.52,-0.87,0
496,10.1016/j.evalprogplan.2015.03.001,Evaluation of competence-based teaching in hig...,Competence-based teaching in higher education ...,Competence-based teaching; Higher education; I...,curriculum; European Union; higher education; ...,Evaluation of competence-based teaching in hig...,"[evalu, compet, base, teach, higher, educ, the...",538,bergsmann e. (2015),<table><tr><td>Authors:</td><td>bergsmann e.; ...,...,"{'Pub. year': 2015.0, 'Citations': 58.0, 'Norm...",10.92,-9.19,3.33,6.53,-1.66,0.60,1.64,1.54,0
498,10.1016/j.tate.2015.04.001,A phenomenographic analysis of the implementat...,After one decade of Competence-Based Education...,Assessment; Competence-based education; Higher...,,A phenomenographic analysis of the implementat...,"[phenomenograph, analysi, implement, compet, b...",545,koenen a.-k. (2015),<table><tr><td>Authors:</td><td>koenen a.-k.; ...,...,"{'Pub. year': 2015.0, 'Citations': 48.0, 'Norm...",3.30,-2.56,0.28,1.64,1.32,2.47,-0.22,2.24,0
499,10.15516/cje.v17i2.1062,Transversal competences of university students...,One of the key changes of the European Higher ...,Competence based education; Engineering educat...,,Transversal competences of university students...,"[transvers, compet, univers, student, engin, t...",546,hernandez-linares r. (2015),<table><tr><td>Authors:</td><td>hernandez-lina...,...,"{'Pub. year': 2015.0, 'Citations': 6.0, 'Norm....",5.48,-2.78,3.12,2.26,-1.99,3.48,2.24,-0.72,0
511,10.1080/02671522.2012.745895,The change towa

In [219]:
table_hist_per_topic = pd.DataFrame(df_topic["Topic"].value_counts()).reset_index().rename(columns={"index": "Topic", "Topic": "Frequency"})
table_hist_per_topic["Topic"] = table_hist_per_topic["Topic"].astype(str)
table_hist_per_topic = table_hist_per_topic.sort_values("Topic")

fig = px.bar(table_hist_per_topic, color="Topic", x="Topic", y="Frequency", title=f"Frecuency of topics (Cluster {cluster_id})")
fig.update_layout(
    font=dict(
        size=18
    )
)
fig.show()

In [220]:
relevant_topics = table_hist_per_topic["Topic"].values

In [221]:
data_strengths_per_topic = lsi.print_topics(num_topics)

In [222]:
strengths_per_topic = []
for topic_k in range(len(data_strengths_per_topic)):
    id_k, strengths_k = data_strengths_per_topic[topic_k]
    strengths_k = strengths_k.split(" + ")
    for a_w in strengths_k:
        a, w = a_w.split("*")
        a = float(a)
        w=w.replace('"', "")
        o = {"Topic": str(id_k), "Term": w, "Strength": a}
        strengths_per_topic.append(o)

In [223]:
table_strength = pd.DataFrame(strengths_per_topic)
table_strength = table_strength[table_strength["Topic"].isin(relevant_topics)]


fig = px.bar(table_strength, color="Topic", x="Term", y="Strength", title=f"Strength of topics terms (Cluster {cluster_id})")
fig.update_layout(
    font=dict(
        size=18
    )
)
fig.show()